# Import Packages

In [ ]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

from functionalities import filemanager as fm
from functionalities import dataloader as dl
from tqdm import tqdm_notebook as tqdm
from functionalities import loss as lo
from functionalities import gpu 
from functionalities import plot as pl

from architecture import CIFAR_autoencoder as cifar

# Pretrainin Setup

In [ ]:
num_epochs = 100
batch_size = 128
learning_rate = 1e-3
milestones = [60, 85, 100]
number_dev = 5

device = gpu.get_device(number_dev)

if not os.path.exists('./dc_img'):
    os.mkdir('./dc_img')

def to_img(x):
    #x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 3, 32, 32)
    return x

In [ ]:
trainset, testset, classes = dl.load_cifar()
trainloader, validloader, testloader = dl.make_dataloaders(trainset, testset, batch_size)

# Training 

In [ ]:
bottleneck_train_log = []
bottleneck_test_log = []    

for bottleneck in [2 ** x for x in range(11)]:
    print('bottleneck dimension: {}'.format(bottleneck))
    model = cifar.cifar_autoencoder(bottleneck).cuda()
    #criterion = nn.MSELoss() # l2 loss
    #criterion = nn.L1Loss() # l1 loss
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=0.1)
    
    train_loss_log = []
    test_loss_log = []

    for epoch in range(num_epochs):
        scheduler.step()
        for data in tqdm(trainloader):
            img, _ = data
            img = Variable(img).cuda()
            # ===================forward=====================
            output = model(img)
            loss = lo.l1_loss(output, img)
            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # ===================log========================
        train_loss_log.append(loss.data.item())
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.data.item()))
        if epoch % 10 == 0 or epoch == (num_epochs - 1):
            pic = to_img(output.cpu().data)
            save_image(pic, './dc_img/l1_cifar_image_{}_{}.png'.format(bottleneck, epoch))
            
        with torch.no_grad():
            test_loss = 0
            for data in tqdm(testloader):
                img, _ = data
                img = Variable(img).cuda()
                output = model(img)
                test_loss += lo.l1_loss(output, img).data.item()
            
            test_loss /= len(testloader)
            test_loss_log.append(test_loss)
            print('test loss:{:.4f}'.format(test_loss))
            
    bottleneck_train_log.append(train_loss_log[-1])
    bottleneck_test_log.append(test_loss_log[-1])    

    torch.save(model.state_dict(), './l1_cifar_autoencoder_{}.pth'.format(bottleneck))
    fm.save_model(model, 'l1_cifar_autoencoder_{}'.format(bottleneck))
    fm.save_weight(model, 'l1_cifar_autoencoder_{}'.format(bottleneck))
    fm.save_variable([train_loss_log, test_loss_log], 'l1_cifar_autoencoder_{}'.format(bottleneck))
    
fm.save_variable([bottleneck_train_log, bottleneck_test_log], 'l1_cifar_autoencoder_bottleneck')

# Plot Reconstruction and Difference Images Examples

In [ ]:
from torch.autograd import Variable
import torchvision
import matplotlib.pyplot as plt

num_img = 100
grid_row_size = 10

img, label = next(iter(testloader))
#img = img.view(img.size(0), -1)
img = Variable(img).cuda()

for i in [2 ** x for x in range(11)]:
    print('bottleneck dimension: {}'.format(i))
    model = fm.load_model('l1_cifar_autoencoder_{}'.format(i)).to(device)
    output = model(img.to(device))

    original = to_img(img.cpu().data) 
    pic = to_img(output.cpu().data)

    print("Original Image:")
    pl.imshow(torchvision.utils.make_grid(original[:num_img].detach(), grid_row_size), filename="com_classic_cifar_{}_original".format(i))
    print("Reconstructed Image:")
    pl.imshow(torchvision.utils.make_grid(pic[:num_img].detach(), grid_row_size), filename="com_classic_cifar_{}_reconstructed".format(i))
    print("Difference:")
    diff_img = (original - pic + 1) / 2
    pl.imshow(torchvision.utils.make_grid(diff_img[:num_img].detach(), grid_row_size), filename="com_classic_cifar_{}_difference".format(i))

# Plot Recontruction Loss against Bottleneck Size

In [ ]:
train, test = fm.load_variable("l1_cifar_autoencoder_bottleneck")
y = [train, test]
x = []

pl.plot([2 ** x for x in range(11)], y, 'latent dimension', 'loss', ['train', 'test'], 'Train & Test Reconstruction Loss History', 'loss_l1_cifar_lin_bottleneck') 